In [84]:
# download to local
!dx download --lightweight -fr data

FileNotFoundError: [Errno 2] No such file or directory


In [ ]:
import os
import pandas as pd
#get list for files have not been download successfully
#df =  pd.read_csv("file_paths.txt", header=None)
result_file_path = os.path.join('/Users/xiaoqianxiao/UKB/data/derivatives', 'output.txt')
df =  pd.read_csv("file_paths.txt")
df['filename'] = df['filepath'].str.split(r'[/]', expand=True)[5]
directory = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
file_names = os.listdir(directory)
df['done_files'] = df['filename'].apply(lambda x: (x in file_names)).replace((1,0),('True','False'))
df.loc[df['done_files'] == False]['filepath'].to_csv('file_paths_remain.txt', sep='\t', index=False, header=False)
df.loc[df['done_files'] == False]['filename'].to_csv(result_file_path, sep='\t', index=False, header=False)

In [56]:
def unzip_files(source_path, unzip_path):
    # Change to the download directory
    os.chdir(download_path)
    # Iterate over all .zip files in the directory
    for f in os.listdir(download_path):
        if f.endswith('.zip'):
            # Create a directory for the unzipped contents
            destination = os.path.join(unzip_path, f[:-4])  # Remove ".zip" from filename
            os.makedirs(destination, exist_ok=True)
            
            # Unzip the file into the destination directory
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(destination)
            print(f"Extracted {f} to {destination}")
            
download_path = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
unzip_path = '/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB'
unzip_files(download_path, unzip_path)

Extracted 4023927_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/4023927_31022_2_0
Extracted 3037800_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3037800_31022_2_0
Extracted 1309909_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/1309909_31022_2_0
Extracted 1838754_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/1838754_31022_2_0
Extracted 1276018_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/1276018_31022_2_0
Extracted 5461097_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5461097_31022_2_0
Extracted 5549757_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5549757_31022_2_0
Extracted 5857506_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5857506_31022_2_0
Extracted 5597280_31022_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5597280_31022_2_0
Extracted 3721874_31022_2_0.zip to /Users/xiao

In [4]:
#transform to BIDS format
import os
import re
import shutil
# Function to rename files to BIDS format
#session: Instance(2=Imaging visit; 3=First repeat imaging visit)
def rename_to_bids(base_dir,target_dir):
    # Loop through each subdirectory in the base directory
    for subject_dir in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_dir)

        if os.path.isdir(subject_path):
            # Extract the subject ID from the folder name
            subject_id = re.split(r'[_\s;]+', subject_dir)[0]
            session_id =re.split(r'[_\s;]+', subject_dir)[2]
        
            # Loop through the files in the subject directory
            for filename in os.listdir(subject_path):
                if filename.endswith('.csv.gz'):
                    space_id = re.split(r'[.\s;]+', filename)[1]
                    #sub-subID_ses-mri_run-runID_task-{task_name}_space-space.csv.gz
                    task_name = "rest"  # Modify this if you have different tasks
                    new_filename = f"sub-{subject_id}_ses-{session_id}_task-{task_name}_space-{space_id}.csv.gz"
                    new_file_path = os.path.join(target_dir, new_filename)

                    # Full path of the old file
                    old_file_path = os.path.join(subject_path, filename)

                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    #shutil.copy(old_file_path, new_file_path)
                    #print(f"Renamed: {old_file_path} to {new_file_path}")

# Set the path to your directory containing the files
# base_dir_parent = "/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB"
# dict_data_sets = get_all_folders(base_dir_parent)
# for folder in dict_data_sets.keys():
#     base_dir = dict_data_sets[folder]
#     target_dir = os.path.join("/Users/xiaoqianxiao/UKB/data/derivatives/timeseries",folder)
#     if not os.path.exists(target_dir):
#         os.makedirs(target_dir)
#     # Call the function
#     rename_to_bids(base_dir, target_dir)

In [57]:
import os
import re
import gzip
import shutil

def rename_to_bids(base_dir, target_dir, compress=False):
    os.makedirs(target_dir, exist_ok=True)

    for subject_dir in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_dir)
        if not os.path.isdir(subject_path):
            continue

        # Extract subject/session IDs safely
        parts_sub = re.split(r'[_\s;]+', subject_dir)
        subject_id = parts_sub[0]
        session_id = parts_sub[2] if len(parts_sub) > 2 else "01"  # fallback if missing

        # Loop through CSV files
        for filename in os.listdir(subject_path):
            if not filename.endswith('.csv'):
                continue

            base = os.path.basename(filename)
            name, ext = os.path.splitext(base)
            parts = name.split("_")

            if len(parts) >= 3:
                metric = "".join(parts[1:-1])  # collapse underscores
                streamlines = parts[-1]

                # Build new filename
                new_filename = (
                    f"sub-{subject_id}_ses-{session_id}_"
                    f"metrics-{metric}_streamlines-{streamlines}{ext}"
                )
                if compress:
                    new_filename += ".gz"

                old_file_path = os.path.join(subject_path, filename)
                new_file_path = os.path.join(target_dir, new_filename)

                if compress:
                    with open(old_file_path, "rb") as f_in:
                        with gzip.open(new_file_path, "wb") as f_out:
                            shutil.copyfileobj(f_in, f_out)
                else:
                    shutil.copy2(old_file_path, new_file_path)

                print(f"Renamed: {old_file_path} → {new_file_path}")

# Example usage
base_dir = "/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB"
target_dir = "/Users/xiaoqianxiao/UKB/data/derivatives/timeseries"
rename_to_bids(base_dir, target_dir, compress=False)


Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3335361_31022_2_0/connectome_mean_length_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-3335361_ses-2_metrics-meanlength_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3335361_31022_2_0/connectome_mean_FA_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-3335361_ses-2_metrics-meanFA_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3335361_31022_2_0/connectome_streamline_count_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-3335361_ses-2_metrics-streamlinecount_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3335361_31022_2_0/connectome_sift2_fbc_10M.csv → /Users/xiaoqianxiao/UKB/data/derivatives/timeseries/sub-3335361_ses-2_metrics-sift2fbc_streamlines-10M.csv
Renamed: /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/1683470_31022_2_0/connectome_mean_length_10M.csv → /